<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# Testing of define_model

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import logging
import tensorflow as tf

from zoobot.tensorflow.estimators import define_model

2022-10-21 11:22:47.030967: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from zoobot.tensorflow.training import train_with_keras

In [6]:
import argparse
import os

import wandb
from sklearn.model_selection import train_test_split

from pytorch_galaxy_datasets.prepared_datasets import decals_dr5_setup

from zoobot.shared import label_metadata, schemas
from zoobot.tensorflow.training import train_with_keras

In [7]:
question_answer_pairs = label_metadata.decals_dr5_ortho_pairs  # dr5
dependencies = label_metadata.decals_ortho_dependencies
schema = schemas.Schema(question_answer_pairs, dependencies)
logging.info('Schema: {}'.format(schema))

data_dir = './zoo_data'

# use the setup() methods in pytorch_galaxy_datasets.prepared_datasets to get the canonical (i.e. standard) train and test catalogs
canonical_train_catalog, _ = decals_dr5_setup(root=data_dir, train=True, download=False)
canonical_test_catalog, _ = decals_dr5_setup(root=data_dir, train=False, download=False)

In [8]:
checkpoint_path = './vit_2xgpu_lowadam/checkpoint'

In [9]:
# checkpoint reader
reader = tf.train.load_checkpoint(checkpoint_path)

shape_from_key = reader.get_variable_to_shape_map()
dtype_from_key = reader.get_variable_to_dtype_map()

sorted(shape_from_key.keys())

['_CHECKPOINTABLE_OBJECT_GRAPH',
 'layer_with_weights-0/_blocks/0/0/_block_attention/_k_proj/bias/.ATTRIBUTES/VARIABLE_VALUE',
 'layer_with_weights-0/_blocks/0/0/_block_attention/_k_proj/bias/.OPTIMIZER_SLOT/optimizer/m/.ATTRIBUTES/VARIABLE_VALUE',
 'layer_with_weights-0/_blocks/0/0/_block_attention/_k_proj/bias/.OPTIMIZER_SLOT/optimizer/v/.ATTRIBUTES/VARIABLE_VALUE',
 'layer_with_weights-0/_blocks/0/0/_block_attention/_k_proj/weight/.ATTRIBUTES/VARIABLE_VALUE',
 'layer_with_weights-0/_blocks/0/0/_block_attention/_k_proj/weight/.OPTIMIZER_SLOT/optimizer/m/.ATTRIBUTES/VARIABLE_VALUE',
 'layer_with_weights-0/_blocks/0/0/_block_attention/_k_proj/weight/.OPTIMIZER_SLOT/optimizer/v/.ATTRIBUTES/VARIABLE_VALUE',
 'layer_with_weights-0/_blocks/0/0/_block_attention/_o_proj/bias/.ATTRIBUTES/VARIABLE_VALUE',
 'layer_with_weights-0/_blocks/0/0/_block_attention/_o_proj/bias/.OPTIMIZER_SLOT/optimizer/m/.ATTRIBUTES/VARIABLE_VALUE',
 'layer_with_weights-0/_blocks/0/0/_block_attention/_o_proj/bias/.OPT

In [10]:
model = define_model.get_model(34, 300, 224, 224, maxvit_name='MaxViTTiny', use_effnet=False)

In [11]:
model.load_weights(checkpoint_path).expect_partial()

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 perma_random_rotation (Perm  (None, 300, 300, 1)      0         
 aRandomRotation)                                                
                                                                 
 perma_random_flip (PermaRan  (None, 300, 300, 1)      0         
 domFlip)                                                        
                                                                 
 perma_random_crop (PermaRan  (None, 224, 224, 1)      0         
 domCrop)                                                        
                                                                 
 maxvit (MaxViT)             (None, 1280)              31106632  
                                                                 
 top_dropout (PermaDropout)  (None, 1280)              0         
                                                        

In [14]:
from zoobot.tensorflow.predictions import predict_on_dataset
from zoobot.tensorflow.data_utils.image_datasets import get_image_dataset

In [15]:
paths = canonical_train_catalog.sample(100).reset_index(drop=True)['file_loc']


In [16]:
initial_size = 300
batch_size = 4

In [17]:
# Load the images as a tf.data.Dataset, just as for training

raw_image_ds = get_image_dataset(paths, 'jpg', initial_size, batch_size)

In [18]:
for batch in raw_image_ds.take(1):
    print(batch['matrix'].numpy().shape)
#     print(images.shape, labels.shape)

(4, 300, 300, 3)


In [19]:
from zoobot.tensorflow.estimators import preprocess

In [20]:
preprocessing_config = preprocess.PreprocessingConfig(label_cols=[], 
                                                      input_size=initial_size, 
                                                      make_greyscale=True)

image_ds = preprocess.preprocess_dataset(raw_image_ds, preprocessing_config)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [27]:
answer_strs = [answer.text for answer in schema.answers]

In [21]:
for images, labels in image_ds.take(1):
    print(images.numpy().shape)
#     print(images.shape, labels.shape)

(4, 300, 300, 1)


In [22]:
# Finally predict
preds, ids = predict_on_dataset.predict(
    ds=image_ds,
    model=model,
    n_samples=1,  # number of dropout forward passes
    label_cols=answer_strs,  # used for output csv header only
    save_loc='./vit_2xgpu_lowadam/prediction_output.csv'
)

2022-10-21 11:24:19.947308: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: sequential/perma_random_crop/loop_body/cond/branch_executed/_256


25/25 [==============================] - 28s 885ms/step


In [24]:
preds.shape

(100, 34, 1)

In [28]:
answer_strs[:3]

['smooth-or-featured-dr5_smooth',
 'smooth-or-featured-dr5_featured-or-disk',
 'smooth-or-featured-dr5_artifact']

In [26]:
preds[:10,:3,0]

array([[ 6.62341  ,  2.5621195,  3.867976 ],
       [65.290985 ,  6.706959 ,  7.336084 ],
       [23.132849 ,  5.670578 ,  1.8219006],
       [13.310388 ,  7.86274  ,  1.5656048],
       [33.754642 ,  7.0516315,  2.341342 ],
       [ 2.8688004, 18.43005  ,  1.1450509],
       [77.10675  ,  5.180824 ,  6.7365575],
       [42.9265   , 11.399221 ,  2.5153685],
       [ 4.6653786, 11.475128 ,  1.3260076],
       [ 9.682583 ,  8.776474 ,  1.639996 ]], dtype=float32)

In [43]:
ids[1]

'/Users/nicolasgalvarinoguerravaras/Documents/AstroHackWeek2022/zoo_data/images/J123/J123915.53+095514.8.jpg'

In [38]:
ids[5]

'/Users/nicolasgalvarinoguerravaras/Documents/AstroHackWeek2022/zoo_data/images/J003/J003807.42-023132.5.jpg'

In [31]:
from PIL import Image

In [35]:
galaxy_id = 5

In [41]:
featured = 5
Image.open(ids[featured]).show()

In [42]:
smooth = 1
Image.open(ids[smooth]).show()

In [37]:
galaxy = preds[galaxy_id, :, 0]
galaxy[0] / galaxy[:3].sum()

0.12782094